### Importing the libraries 

In [8]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
import os

In [10]:
# Loading environment variables from .env file
load_dotenv()

# Getting the API key from environment variables
api_key = os.getenv('IATI_API_KEY')

# API endpoint
url = 'https://api.iatistandard.org/datastore/activity/iati'

# parameters 
params = {
    'q': '*:*',  # Query all records
    'rows': 5    # Limit the number of results
}

# headers
headers = {
    'Accept': 'application/json',
    'Ocp-Apim-Subscription-Key': api_key 
}

# Making the GET request with parameters and headers
response = requests.get(url, params=params, headers=headers)

# response content for debugging
print(response.content)

if response.status_code == 200:
    try:
        # Parsing the XML response
        root = ET.fromstring(response.content)
        
        # Extracting data from XML
        activities = []
        for activity in root.findall('.//iati-activity'):
            activity_data = {
                'identifier': activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None,
                'title': activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None,
                'description': activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None,
                'status': activity.find('.//activity-status').attrib.get('code') if activity.find('.//activity-status') is not None else None,
                'start_date': activity.find('.//activity-date[@type="1"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="1"]') is not None else None,
                'end_date': activity.find('.//activity-date[@type="4"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="4"]') is not None else None,
                'reporting_org': activity.find('.//reporting-org/narrative').text if activity.find('.//reporting-org/narrative') is not None else None,
                'recipient_country': activity.find('.//recipient-country/narrative').text if activity.find('.//recipient-country/narrative') is not None else None,
                'sector': activity.find('.//sector/narrative').text if activity.find('.//sector/narrative') is not None else None,
                'budget': activity.find('.//budget/value').text if activity.find('.//budget/value') is not None else None,
                'transaction_type': activity.find('.//transaction/transaction-type').attrib.get('code') if activity.find('.//transaction/transaction-type') is not None else None,
                'transaction_date': activity.find('.//transaction/transaction-date').attrib.get('iso-date') if activity.find('.//transaction/transaction-date') is not None else None,
                'transaction_value': activity.find('.//transaction/value').text if activity.find('.//transaction/value') is not None else None,
                'transaction_description': activity.find('.//transaction/description/narrative').text if activity.find('.//transaction/description/narrative') is not None else None,
                'provider_org': activity.find('.//transaction/provider-org/narrative').text if activity.find('.//transaction/provider-org/narrative') is not None else None,
                'receiver_org': activity.find('.//transaction/receiver-org/narrative').text if activity.find('.//transaction/receiver-org/narrative') is not None else None,
                'finance_type': activity.find('.//transaction/finance-type').attrib.get('code') if activity.find('.//transaction/finance-type') is not None else None
            }
            activities.append(activity_data)
        
        # Converting to DF
        df = pd.DataFrame(activities)
        display(df)
    except ET.ParseError as e:
        print("Error parsing XML:", e)
else:
    print(f"Error: {response.status_code}")


b'<?xml version="1.0" encoding="UTF-8"?><iati-activities generated-datetime="2024-07-20T23:49:05.909Z" version="2.03"><iati-activity last-updated-datetime="2024-06-13T17:35:50" xml:lang="EN" default-currency="USD" hierarchy="02">\n       <iati-identifier>XI-IATI-WBTF-P104385</iati-identifier>\n       <reporting-org ref="XI-IATI-WBTF" type="40">\n       <narrative>World Bank Trust Funds</narrative>\n       </reporting-org>\n       <title>\n       <narrative>Institutional Strengthening of the Legal Claims Management System</narrative>\n       </title>\n       <description type="1">\n       <narrative> The overall development objective of the proposed IDF grant would be to strengthen the ability of the Ministry of Interior and Justice (MIJ) to promote alternative dispute resolution (ADR) and to effectively coordinate, monitor and control potential high-impact contractual and labor legal claims through appropriate case-tracking systems and capacity-building models. Specific objectives are:

,identifier,title,description,status,start_date,end_date,reporting_org,recipient_country,sector,budget,transaction_type,transaction_date,transaction_value,transaction_description,provider_org,receiver_org,finance_type
0,XI-IATI-WBTF-P104385,Institutional Strengthening of the Legal Claim...,The overall development objective of the prop...,3,2006-04-27,2011-04-04,World Bank Trust Funds,COLOMBIA,+![cdata[Anti-corruption organisations and ins...,478491.39,11,2020-06-30,478491.39,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),COLOMBIA,110
1,XI-IATI-WBTF-P104395,Legal Education Capacity Building Project,This IDF will suport the Nigeria Legal Educait...,3,2007-06-20,2011-11-22,World Bank Trust Funds,NIGERIA,+![cdata[Legal and judicial development]]+,440062.74,11,2020-06-30,440062.74,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),NIGERIA,110
2,XI-IATI-WBTF-P104397,HN Enhancing Transparency and Accountability i...,The Transparency law that was approved on Nove...,3,2007-04-19,2011-09-08,World Bank Trust Funds,HONDURAS,+![cdata[Anti-corruption organisations and ins...,332717.02,11,2020-06-30,332717.02,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),HONDURAS,110
3,XI-IATI-WBTF-P104398,Strengthening Monitoring and Evaluation in Brazil,Soound monitoring and evaluation of programs a...,3,2008-03-26,2011-08-26,World Bank Trust Funds,BRAZIL,+![cdata[Public sector policy and administrati...,155000.00,11,2020-06-30,155000.00,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),BRAZIL,110
4,XI-IATI-WBTF-P104400,HT - Improving Health Surveillance,Capacity Building of Ministry of Health on He...,3,2009-04-29,2011-10-11,World Bank Trust Funds,HAITI,+![cdata[Health policy and administrative mana...,93212.00,11,2020-06-30,93212.00,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),HAITI,110
